In [4]:
import os
import sys
import pandas as pd
from datasets import Dataset
from tqdm.notebook import tqdm
from read_data import get_essay_sets, essays_by_set
from transformers import BertForSequenceClassification, BertTokenizer
import torch

In [5]:
data_directory = 'test_data_predictions.csv'
test_set = pd.read_csv(data_directory)

In [8]:
def score_essay_benchmark(test_set, essay_text):
    
    unique_labels = len(test_set['Score1'].unique())
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=unique_labels)
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    inputs = tokenizer(
        essay_text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_score = torch.argmax(logits, dim=-1).item()
    return predicted_score


In [ ]:
tqdm.pandas()
test_set['benchmark_prediction'] = test_set.progress_apply(lambda x: score_essay_benchmark(test_set, x['EssayText']), axis=1)

  0%|          | 0/5224 [00:00<?, ?it/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are